In [1]:
import copy
import os.path
import sys

pwd = os.getcwd()
sys.path.append(os.path.dirname(pwd))

from utils.model_utils.evaluate import evaluate_model
from utils.model_utils.load_model import *
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_data
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from utils.model_utils.save_module import save_module
from datetime import datetime
from utils.decompose_utils.concern_modularization import ConcernModularizationBert
from utils.decompose_utils.sampling import sampling_class
from utils.dataset_utils.load_dataset import convert_dataset_labels_to_binary, extract_and_convert_dataloader
import torch


In [2]:
model_name = "sadickam/sdg-classification-bert"
model_type = "pretrained"
data = "OSDG"
num_labels = 16


# model_name = "textattack/bert-base-uncased-imdb"
# model_type = "pretrained"
# data = "IMDb"
# num_labels = 2

# model_name = "fabriceyhc/bert-base-uncased-yahoo_answers_topics"
# model_type = "pretrained"
# data = "Yahoo"
# num_labels = 10

In [3]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

checkpoint_name = None
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model_config = ModelConfig(
    _model_name=model_name,
    _model_type=model_type,
    _data=data,
    _transformer_config=config,
    _checkpoint_name=checkpoint_name,
    _device=device,
)


In [4]:
for i in range(num_labels):
    model, tokenizer, checkpoint = load_classification_model(model_config, train_mode=False)

    train_dataloader, valid_dataloader, test_dataloader = load_data(
        model_config, batch_size=32, test_size=0.3
    )

    print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    all_samples = sampling_class(
        train_dataloader, 200, 20, num_labels, False, 4, device=device
    )

    print("origin")
    evaluate_model(model, model_config, test_dataloader)

    module = copy.deepcopy(model)
    wr = WeightRemoverBert(model, p=0.9)
    ci = ConcernIdentificationBert(model, p=0.4)
    ti = TanglingIdentification(model, p=0.5)

    print("Start Positive CI sparse")

    eval_step = 5
    for idx, batch in enumerate(all_samples):
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            wr.propagate(module, input_ids)
        if idx % eval_step:
            # result = evaluate_model(module, model_config, test_dataloader)
            pass

    print("Start Positive CI after sparse")

    for idx, batch in enumerate(positive_samples):
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ci.propagate(module, input_ids)
        if idx % eval_step:
            # result = evaluate_model(module, model_config, test_dataloader)
            pass

    print("Start Negative TI")

    for idx, batch in enumerate(negative_samples):
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            ti.propagate(module, input_ids)
        if idx % eval_step:
            # result = evaluate_model(module, model_config, test_dataloader)
            pass

    ConcernModularizationBert.channeling(module, ci.active_node, ti.dead_node, i, model_config.device)
    binary_module = ConcernModularizationBert.convert2binary(model_config, module)
    # save_module(binary_module, model_config.module_dir, model_config.model_name)
    
    for m in range(num_labels):
        if i == m:
            continue
        print(f"[{i}, {m}]")
        converted_test_dataloader = extract_and_convert_dataloader(test_dataloader, i, m)
        result = evaluate_model(module, model_config, converted_test_dataloader)


Directory /home/Minwoo/LESN/Decompose/DecomposeBERT/Models/Configs/pretrained/sadickam/sdg-classification-bert exists.
Loading the model.
Start Time:01:20:47
#Module 0 in progress....
origin


Evaluating: 100%|██████████| 400/400 [04:02<00:00,  1.65it/s]


Loss: 0.9480
Precision: 0.7801, Recall: 0.7867, F1-Score: 0.7793
              precision    recall  f1-score   support

           0       0.77      0.66      0.71       797
           1       0.84      0.72      0.78       775
           2       0.88      0.87      0.88       795
           3       0.87      0.83      0.85      1110
           4       0.86      0.80      0.83      1260
           5       0.88      0.69      0.77       882
           6       0.85      0.80      0.83       940
           7       0.49      0.61      0.54       473
           8       0.66      0.85      0.74       746
           9       0.62      0.73      0.67       689
          10       0.75      0.79      0.77       670
          11       0.62      0.81      0.70       312
          12       0.73      0.81      0.77       665
          13       0.83      0.85      0.84       314
          14       0.85      0.78      0.81       756
          15       0.97      0.98      0.97      1607

    accuracy   

Evaluating: 100%|██████████| 50/50 [00:25<00:00,  1.96it/s]


Loss: 0.4574
Precision: 0.8177, Recall: 0.7748, F1-Score: 0.7689
              precision    recall  f1-score   support

           0       0.93      0.59      0.72       775
           1       0.71      0.95      0.81       797

    accuracy                           0.78      1572
   macro avg       0.82      0.77      0.77      1572
weighted avg       0.82      0.78      0.77      1572

[0, 2]


...

Loss: 0.0415
Precision: 0.9892, Recall: 0.9874, F1-Score: 0.9883
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       665
           1       0.99      0.99      0.99      1607

    accuracy                           0.99      2272
   macro avg       0.99      0.99      0.99      2272
weighted avg       0.99      0.99      0.99      2272

[15, 13]


Evaluating: 100%|██████████| 61/61 [00:25<00:00,  2.37it/s]


Loss: 0.5702
Precision: 0.8025, Recall: 0.5418, F1-Score: 0.5375
              precision    recall  f1-score   support

           0       0.76      0.09      0.16       314
           1       0.85      0.99      0.92      1607

    accuracy                           0.85      1921
   macro avg       0.80      0.54      0.54      1921
weighted avg       0.83      0.85      0.79      1921

[15, 14]


Evaluating: 100%|██████████| 74/74 [00:33<00:00,  2.23it/s]

Loss: 0.3633
Precision: 0.8463, Recall: 0.6222, F1-Score: 0.6218
              precision    recall  f1-score   support

           0       0.95      0.25      0.40       756
           1       0.74      0.99      0.85      1607

    accuracy                           0.76      2363
   macro avg       0.85      0.62      0.62      2363
weighted avg       0.81      0.76      0.70      2363

